In [1]:
import os

In [2]:
%pwd

'/Users/ranvirmakker/Desktop/chicken-disease-project/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/ranvirmakker/Desktop/chicken-disease-project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: Path
    local_data_file: Path
    unzip_dir: Path

In [ ]:
from src.ChickenDisease.constants import *
from src.ChickenDisease.utils.common import read_yaml,create_directories


In [15]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


In [16]:
import os
import urllib.request as request
import zipfile
from src.ChickenDisease import logger
from src.ChickenDisease.utils.common import get_size

In [17]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        """Downloads file from URL if it doesn't already exist"""
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self) -> None:
        """Extracts the zip file into the data directory"""
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"Extracted zip file to: {unzip_path}")

In [18]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.error(f"Error during data ingestion: {e}")
    raise e

[2025-08-06 21:50:12,114: INFO: common:YAML file loaded: config/config.yaml]
[2025-08-06 21:50:12,117: INFO: common:YAML file loaded: params.yaml]
[2025-08-06 21:50:12,118: INFO: common:Created directory: artifacts]
[2025-08-06 21:50:12,118: INFO: common:Created directory: artifacts/data_ingestion]
[2025-08-06 21:50:15,357: INFO: 1838936781:artifacts/data_ingestion/data.zip downloaded! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 5E11:3F76E3:1636B2:332D29:689380BC
Accept-Ranges: bytes
Date: Wed, 06 Aug 2025 16:20:14 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10227-MAA
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1754